In [2]:
from flows import api
import requests
import json
import pandas as pd
'''
  code author: Emir Karamehmetoglu (emir.k@phys.au.dk)
  
Query ZTF target information using alerce.
https://alerceapi.readthedocs.io/en/latest/ztf_db.html

'''

cat = api.get_catalog(8, output='json')
tgt = cat['target']

query={
    "records_per_pages":20,
    "query_parameters": {
      "coordinates":{
        "ra":  ra,
        "dec": dec,
        "sr": 0.1 
    }
    },
    "total":5
}

url='http://ztf.alerce.online/query'
response=requests.post(url,json=query)

print("Status code: ", response.status_code)
print("Printing Entire Post Request")
print(response.json())

Status code:  200
Printing Entire Post Request
{'total': 5, 'num_pages': 1, 'page': 1, 'result': {'ZTF20abbplei': {'oid': 'ZTF20abbplei', 'nobs': 71, 'mean_magap_g': 18.9711868888453, 'mean_magap_r': 18.3439454165372, 'median_magap_g': 19.2523002624512, 'median_magap_r': 18.1685009002686, 'max_magap_g': 21.0400009155273, 'max_magap_r': 20.1651000976562, 'min_magap_g': 17.1700992584229, 'min_magap_r': 16.2579002380371, 'sigma_magap_g': 0.994304057038332, 'sigma_magap_r': 0.875419127536384, 'last_magap_g': 20.6448001861572, 'last_magap_r': 19.3416004180908, 'first_magap_g': 19.6744003295898, 'first_magap_r': 19.4298992156982, 'mean_magpsf_g': 19.0063585984079, 'mean_magpsf_r': 18.2000714504358, 'median_magpsf_g': 19.297905921936, 'median_magpsf_r': 18.1164665222168, 'max_magpsf_g': 20.4988994598389, 'max_magpsf_r': 19.6817970275879, 'min_magpsf_g': 17.3469772338867, 'min_magpsf_r': 16.9846000671387, 'sigma_magpsf_g': 0.986631749159696, 'sigma_magpsf_r': 0.69605792741681, 'last_magpsf_g':

In [13]:
type(response.status_code)

int

In [21]:
if response.status_code == 200:
    df=response.json()
    names=list(df['result'].keys())
    oid=df['result'][names[0]]['oid']

In [23]:
url='http://ztf.alerce.online/get_detections'
response=requests.post(url,json={'oid':oid})

#url='http://ztf.alerce.online/get_non_detections'


In [26]:
response.json()['result']

{'detections': [{'candid_str': '1240413131515015007',
   'oid': 'ZTF20abbplei',
   'candid': 1240413131515015007,
   'mjd': 58994.4131366001,
   'fid': 1,
   'diffmaglim': 20.8665409088135,
   'magpsf': 19.6894721984863,
   'magap': 19.6744003295898,
   'sigmapsf': 0.103337325155735,
   'sigmagap': 0.185599997639656,
   'ra': 256.7275223,
   'dec': 30.2714587,
   'sigmara': 0.09,
   'sigmadec': 0.09,
   'isdiffpos': 1,
   'distpsnr1': 6.29877662658691,
   'sgscore1': 0.144893005490303,
   'field': 681,
   'rcid': 15,
   'magnr': 17.9820003509521,
   'sigmagnr': 0.0939999967813492,
   'rb': 0.928571403026581,
   'magpsf_corr': 19.6894721984863,
   'magap_corr': 19.6744003295898,
   'sigmapsf_corr': 0.103337325155735,
   'sigmagap_corr': 0.185599997639656,
   'has_stamps': True,
   'parent_candid': -1},
  {'candid_str': '1241276041515015003',
   'oid': 'ZTF20abbplei',
   'candid': 1241276041515015003,
   'mjd': 58995.2760417,
   'fid': 2,
   'diffmaglim': 20.6843776702881,
   'magpsf': 1

In [32]:
det=response.json()['result']['detections']

In [50]:
try:

KeyError: 'oid'

In [41]:
df2=pd.DataFrame(det)

bands={}
bands['g']=df2[['mjd','magpsf_corr','sigmapsf_corr']][df2.fid==1]
bands['r']=df2[['mjd','magpsf_corr','sigmapsf_corr']][df2.fid==2]
if 3 in df2.fid.unique():
    bands[i]=df2[['mjd','magpsf_corr','sigmapsf_corr']][df2.fid==2]
for band in bands:
    bands[band].reset_index(drop=True,inplace=True)
    bands[band]['jd']=bands[band]['mjd']+2400000.5
    bands[band].columns=['mjd','mag','err','jd']
    bands[band].to_csv(tgt['target_name']+'_ztf'+band+'.csv')